In [1]:
from baseline_transformer_POS_conv import *
import copy

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())  # Renvoie True si un GPU est disponible

batch_size=16
epochs=300

True


In [3]:
# Load data using the load_data_1 function
sentences, pos_tags = load_data_1("UD_French-Sequoia/fr_sequoia-ud-train.conllu")

# Create character and tag mappings
char_counts = Counter(char for sentence in sentences for word in sentence for char in word)
char_to_ix = {char: i for i, char in enumerate(char_counts, start=2)}
char_to_ix['<PAD>'], char_to_ix['<UNK>'] = 0, 1  # Padding and unknown character

print(char_to_ix)
tag_counts = Counter(tag for tags in pos_tags for tag in tags)
tag_to_ix = {tag: i for i, tag in enumerate(tag_counts)}
max_word_len = max(len(word) for sentence in sentences for word in sentence)
print(max_word_len)
# Now load the data in the desired format using the load_data function
train_sentences, train_pos_tags = load_data("UD_French-Sequoia/fr_sequoia-ud-train.conllu", char_to_ix, max_word_len)
validation_sentences, validation_pos_tags = load_data("UD_French-Sequoia/fr_sequoia-ud-dev.conllu", char_to_ix, max_word_len)

# Rest of your code for Dataset, DataLoader, Model initialization, etc.

# Dataset and DataLoader
dataset = POSDataset(train_sentences, train_pos_tags, tag_to_ix, max_word_len,char_to_ix)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

validation_dataset = POSDataset(validation_sentences, validation_pos_tags, tag_to_ix, max_word_len,char_to_ix)
validation_data_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


{'g': 2, 'u': 3, 't': 4, 'e': 5, 'n': 6, 'b': 7, 'r': 8, 'c': 9, 'x': 10, 'p': 11, 'o': 12, 's': 13, 'i': 14, 'a': 15, 'd': 16, 'q': 17, 'è': 18, 'l': 19, ',': 20, 'à': 21, 'm': 22, '-': 23, 'é': 24, '.': 25, 'f': 26, 'v': 27, "'": 28, 'j': 29, 'h': 30, '1': 31, '9': 32, '5': 33, '3': 34, 'z': 35, '2': 36, '?': 37, '"': 38, 'ê': 39, 'y': 40, 'ù': 41, 'â': 42, '7': 43, '6': 44, 'û': 45, '0': 46, '4': 47, '%': 48, ';': 49, '8': 50, 'ô': 51, ':': 52, 'k': 53, 'î': 54, 'ç': 55, '!': 56, '(': 57, ')': 58, '/': 59, '°': 60, 'ë': 61, 'w': 62, '=': 63, ' ': 64, '©': 65, '+': 66, '<': 67, '½': 68, 'ä': 69, 'ß': 70, 'ï': 71, '[': 72, ']': 73, '±': 74, 'ö': 75, 'ó': 76, 'á': 77, '&': 78, '^': 79, '$': 80, '<PAD>': 0, '<UNK>': 1}
26


In [4]:

# Model initialization
num_chars = len(char_to_ix)
char_embedding_dim = 512
num_filters = 1024
kernel_size = 6
nhead = 2
nhid = 512
nlayers = 1
tagset_size = len(tag_to_ix)

model = POSTransformerModel(num_chars, char_embedding_dim, num_filters, kernel_size, nhead, nhid, nlayers, tagset_size)

# Loss and Optimizer
loss_function = nn.CrossEntropyLoss(ignore_index=-1)
optimizer = optim.SGD(model.parameters(), lr=0.001)


loss_function = nn.CrossEntropyLoss(ignore_index=-1)
optimizer = optim.Adam(model.parameters(), lr=0.0001)




C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

patience = 20  # Nombre d'époques à attendre après la dernière amélioration de la loss de validation
best_val_accuracy = 0
epochs_no_improve = 0

#Training
epochs=5
for epoch in range(epochs): 
    model.train()
    model.to(device)  # Déplacer le modèle sur le GPU si disponible
    total_loss = 0
    for sentence_in, targets in data_loader:
        sentence_in, targets = sentence_in.to(device), targets.to(device)  # Déplacer les données sur le périphérique
        optimizer.zero_grad()
        tag_scores = model(sentence_in)
        loss = loss_function(tag_scores.view(-1, len(tag_to_ix)), targets.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Utiliser la fonction modifiée pour évaluer la validation loss et l'accuracy
    val_loss, val_accuracy,_,_,_ = evaluate_model(model, validation_data_loader, loss_function,device, tag_to_ix)
    print(f"Epoch {epoch+1}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model = copy.deepcopy(model)
        print(f"best model accuracy: {best_val_accuracy:.4f}")
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    # Arrêt précoce si aucune amélioration
    if epochs_no_improve == patience:
        print("Arrêt précoce : La loss de validation ne s'améliore plus")
        break
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(data_loader)}")


0.6982990025323578
Epoch 1, Validation Loss: 0.4082, Validation Accuracy: 0.8663
best model accuracy: 0.8663
Epoch 1, Loss: 0.7685504098023687
0.7529949540655732
Epoch 2, Validation Loss: 0.2896, Validation Accuracy: 0.9147
best model accuracy: 0.9147
Epoch 2, Loss: 0.3712108508816787
0.7577216670388419
Epoch 3, Validation Loss: 0.2569, Validation Accuracy: 0.9208
best model accuracy: 0.9208
Epoch 3, Loss: 0.28407756920371735
0.7694892546108963
Epoch 4, Validation Loss: 0.2368, Validation Accuracy: 0.9297
best model accuracy: 0.9297
Epoch 4, Loss: 0.2471505442900317
0.8182936151649853
Epoch 5, Validation Loss: 0.2263, Validation Accuracy: 0.9351
best model accuracy: 0.9351
Epoch 5, Loss: 0.21439415918929236


In [6]:
model.eval()
with torch.no_grad():
    # Convert the first sentence in the dataset to character indices
    char_indices = [[char_to_ix.get(char, char_to_ix['<UNK>']) for char in word] for word in sentences[0]]
    char_indices = [word[:max_word_len] + [char_to_ix['<PAD>']] * (max_word_len - len(word)) for word in char_indices]

    # Convert to tensor and add batch dimension
    inputs = torch.tensor(char_indices, dtype=torch.long).unsqueeze(0).to(device)

    # Get tag scores from the model
    tag_scores = model(inputs)
    predicted_tags = [list(tag_to_ix.keys())[tag] for tag in tag_scores[0].argmax(dim=1).cpu()]
    
    print(f"Sentence: {' '.join(sentences[0])}")
    print(f"Predicted POS Tags: {predicted_tags}")
    true_tags = [tag for tag in pos_tags[0]]
    print(f"Vraies étiquettes POS: {true_tags}")



test_sentences, test_pos_tags = load_data("UD_French-Sequoia/fr_sequoia-ud-test.conllu", char_to_ix, max_word_len)


test_dataset = POSDataset(test_sentences, test_pos_tags, tag_to_ix, max_word_len,char_to_ix)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


# Calculer l'accuracy
loss, accuracy, precision, recall, f1 = evaluate_model(model,test_data_loader,loss_function,device,tag_to_ix)
print(f"Test Accuracy : {accuracy:.4f}")
print(f"Test loss : {loss:.4f}")
print(f"Test precision : {precision:.4f}")
print(f"Test recall : {recall:.4f}")
print(f"Test F1 score : {f1:.4f}")

Sentence: gutenberg
Predicted POS Tags: ['PROPN']
Vraies étiquettes POS: ['PROPN']
0.8268819478202635
Test Accuracy : 0.9328
Test loss : 0.2399
Test precision : 0.9959
Test recall : 0.9891
Test F1 score : 0.9925


In [7]:
best_model.eval()
with torch.no_grad():
    # Convert the first sentence in the dataset to character indices
    char_indices = [[char_to_ix.get(char, char_to_ix['<UNK>']) for char in word] for word in sentences[0]]
    char_indices = [word[:max_word_len] + [char_to_ix['<PAD>']] * (max_word_len - len(word)) for word in char_indices]

    # Convert to tensor and add batch dimension
    inputs = torch.tensor(char_indices, dtype=torch.long).unsqueeze(0).to(device)

    # Get tag scores from the model
    tag_scores = best_model(inputs)
    predicted_tags = [list(tag_to_ix.keys())[tag] for tag in tag_scores[0].argmax(dim=1).cpu()]
    
    print(f"Sentence: {' '.join(sentences[0])}")
    print(f"Predicted POS Tags: {predicted_tags}")
    true_tags = [tag for tag in pos_tags[0]]
    print(f"Vraies étiquettes POS: {true_tags}")



test_sentences, test_pos_tags = load_data("UD_French-Sequoia/fr_sequoia-ud-test.conllu", char_to_ix, max_word_len)


test_dataset = POSDataset(test_sentences, test_pos_tags, tag_to_ix, max_word_len,char_to_ix)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


# Calculer l'accuracy
loss, accuracy, precision, recall, f1 = evaluate_model(best_model,test_data_loader,loss_function,device,tag_to_ix)
print(f"Test Accuracy : {accuracy:.4f}")
print(f"Test loss : {loss:.4f}")
print(f"Test precision : {precision:.4f}")
print(f"Test recall : {recall:.4f}")
print(f"Test F1 score : {f1:.4f}")

Sentence: gutenberg
Predicted POS Tags: ['PROPN']
Vraies étiquettes POS: ['PROPN']
0.8268819478202635
Test Accuracy : 0.9328
Test loss : 0.2399
Test precision : 0.9959
Test recall : 0.9891
Test F1 score : 0.9925


In [8]:

best_model.eval()
with torch.no_grad():
    char_indices = [[char_to_ix.get(char, char_to_ix['<UNK>']) for char in word] for word in sentences[0]]
    char_indices = [word[:max_word_len] + [char_to_ix['<PAD>']] * (max_word_len - len(word)) for word in char_indices]
    inputs = torch.tensor(char_indices, dtype=torch.long).unsqueeze(0).to(device)
    tag_scores = best_model(inputs)
    predicted_tags = [list(tag_to_ix.keys())[tag] for tag in tag_scores[0].argmax(dim=1).cpu()]
    print(f"Sentence: {' '.join(sentences[0])}")
    print(f"Predicted POS Tags: {predicted_tags}")
    true_tags = [tag for tag in pos_tags[0]]
    print(f"Vraies étiquettes POS: {true_tags}")

# Évaluation sur l'ensemble de test
test_sentences, test_pos_tags = load_data("UD_French-Sequoia/fr_sequoia-ud-test.conllu", char_to_ix, max_word_len)
test_dataset = POSDataset(test_sentences, test_pos_tags, tag_to_ix, max_word_len, char_to_ix)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Calculer l'accuracy sur l'ensemble de test
loss, accuracy, precision, recall, f1 = evaluate_model(best_model, data_loader, loss_function, device, tag_to_ix)
print(f"Train Accuracy : {accuracy:.4f}")
print(f"Train loss : {loss:.4f}")
print(f"Train precision : {precision:.4f}")
print(f"Train recall : {recall:.4f}")
print(f"Train F1 score : {f1:.4f}")


Sentence: gutenberg
Predicted POS Tags: ['PROPN']
Vraies étiquettes POS: ['PROPN']
0.8374771729428625
Train Accuracy : 0.9523
Train loss : 0.1562
Train precision : 0.9984
Train recall : 0.9938
Train F1 score : 0.9961
